<a href="https://colab.research.google.com/github/Salma-Jamal/Movie-Recommendation-System/blob/main/models/Movie_Recommendation_System_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error
import numpy as np

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/partition500k.csv")

In [ ]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,122,5.0,1996-08-02,Boomerang (1992),Comedy|Romance
1,1,185,5.0,1996-08-02,"Net, The (1995)",Action|Crime|Thriller
2,1,231,5.0,1996-08-02,Dumb & Dumber (1994),Comedy
3,1,292,5.0,1996-08-02,Outbreak (1995),Action|Drama|Sci-Fi|Thriller
4,1,316,5.0,1996-08-02,Stargate (1994),Action|Adventure|Sci-Fi


In [ ]:
# Create a user-item matrix
matrix = data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)


In [ ]:
matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,64508,64614,64620,64716,64839,64906,64957,64969,64997,65133
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3807,0.0,0.0,3.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Calculate the pairwise distances between all items
item_dist = pairwise_distances(matrix.T, metric='cosine')

In [ ]:
# Fit the NearestNeighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(item_dist)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [ ]:
# Use the fitted model to get the k nearest neighbors for each item
neighbors = knn.kneighbors(return_distance=False)

In [ ]:
def RecommendMovies(UserID , neighborsMatrix):
  bestRatedMovie = data[data['userId'] == UserID].loc[:,['movieId',	'rating']].sort_values(by=['rating'])['movieId'].tolist()[-1]
  Nearest_movies = neighborsMatrix[bestRatedMovie]

  for movie_id in Nearest_movies[:10]:
    print(data[data['movieId'] == movie_id ]['title'].tolist()[0])

In [ ]:
RecommendMovies(5,neighbors)

Manon of the Spring (Manon des sources) (1986)
Kull the Conqueror (1997)
Stuart Saves His Family (1995)
Léon: The Professional (Léon) (Professional, The) (1994)
Show, The (1995)
Carlito's Way (1993)
Lawnmower Man 2: Beyond Cyberspace (1996)
Glengarry Glen Ross (1992)
Free Willy 2: The Adventure Home (1995)
My Science Project (1985)


In [ ]:
#Use the k nearest neighbors to predict the ratings for each user-item combination
predictions = matrix.dot(item_dist) / np.array([np.abs(item_dist).sum(axis=1)])

In [ ]:
# Evaluate the accuracy of the predictions
mae = mean_absolute_error(predictions, matrix)
print('MAE:', mae)

MAE: 0.10435287398267933
